In [ ]:
import pandas as pd
df = pd.DataFrame({'sexo': pd.Series(dtype='str'), 'Pts. ': pd.Series(dtype='float')})
tnrs = ['1043688','993751','949798','912620']
for tnr in tnrs:
    url = "https://chess-results.com/tnr{}.aspx?lan=10&zeilen=0&art=1&rd=5&prt=4&excel=2010".format(tnr)
    data = pd.read_excel(url)
    nome_index = data[data.iloc[:, 3] == 'Nome'].index[0]
    first_nan_after_nome = data.iloc[nome_index + 1:, 3].isna().idxmax()
    data = pd.read_excel(url, header=nome_index+1, nrows=first_nan_after_nome-3)
    data = data[['sexo','Pts. ']]
    df = pd.concat([df, data], ignore_index=True)
df['sexo'] = df['sexo'].fillna('M')
df['sexo'] = df['sexo'].apply(lambda x: 'F' if x != 'M' else 'M')
df.columns = ['Sexo', 'Pontuacao']

In [ ]:
import numpy as np
import scipy.stats as stats

# 1. Carregar os dados
jogadoras = df[df['Sexo'] == 'F']
jogadores = df[df['Sexo'] == 'M']

# 2. Criar uma amostra
amostra_f = jogadoras.sample(frac=0.25, random_state=198)  # 25% de jogadoras
amostra_m = jogadores.sample(frac=0.25, random_state=198)  # 25% de jogadores

# 3. Calcular as estatísticas
media_f = amostra_f['Pontuacao'].mean()
media_m = amostra_m['Pontuacao'].mean()
proporcao = media_f / media_m

# 4. Calcular os intervalos de confiança
def intervalo_conf(media, std, n, nivel=0.95):
    z = stats.norm.ppf(1 - (1 - nivel) / 2)
    erro_padrao = std / np.sqrt(n)
    return (media - z * erro_padrao, media + z * erro_padrao)

# Dados da amostra
std_f = amostra_f['Pontuacao'].std()
std_m = amostra_m['Pontuacao'].std()
n_f = amostra_f.shape[0]
n_m = amostra_m.shape[0]

# Intervalos
ic_media_f = intervalo_conf(media_f, std_f, n_f)
ic_media_m = intervalo_conf(media_m, std_m, n_m)
ic_proporcao = intervalo_conf(proporcao, proporcao * (std_f / media_f + std_m / media_m), min(n_f, n_m))

# 5. Comparar com a população
media_pop_f = jogadoras['Pontuacao'].mean()
media_pop_m = jogadores['Pontuacao'].mean()
proporcao_pop = media_pop_f / media_pop_m

print("Média jogadoras (amostra):", media_f, "IC:", ic_media_f)
print("Média jogadores (amostra):", media_m, "IC:", ic_media_m)
print("Proporção (amostra):", proporcao, "IC:", ic_proporcao)
print("\nMédia jogadoras (população):", media_pop_f)
print("Média jogadores (população):", media_pop_m)
print("Proporção (população):", proporcao_pop)
